In [1]:
import os, json
from PIL import Image, ImageDraw, ImageFont
import pandas as pd

In [2]:
def load_config(config_file='config.json'):
    with open(config_file, 'r') as f:
        config = json.load(f)
    return config

config = load_config()

In [3]:
def export_split_image(image_path, output_path_base):
    output_path1 = output_path_base+'left.png'
    output_path2 = output_path_base+'right.png'
    img = Image.open(image_path).convert('RGB')
    width = img.width
    
    # Find where black portion starts
    pixels = img.load()
    black_start = 0
    for y in range(img.height):
        if all(pixels[x, y] == (0,0,0) for x in range(0, width, 10)):  # Sample every 10th pixel
            black_start = y
            break
    
    img1 = img.crop((0, 0, round(width/2), black_start))
    img1.save(output_path1)
    
    img2 = img.crop((round(width/2), 0, width, black_start))
    img2.save(output_path2)

In [4]:
predictions_df = pd.read_csv(config['sample_submission_location']+'SampleSubmission.csv')
predictions_df = predictions_df[predictions_df['id'].str.contains('X_no_damage')]

if not os.path.exists(config['cropped_pairs_location']):
    os.makedirs(config['cropped_pairs_location'])

for idx, row in predictions_df.iterrows():
    id_name = row['id']
    test_image = id_name.replace('_X_no_damage', '')
    test_image_base = test_image.replace('_post_disaster.tif', '')
    image_path = config['test_image_location']+'/'+test_image_base+'_pre_disaster.tif'
    output_path_base = config['cropped_pairs_location']+'/'+test_image_base+'_pre_disaster_'
    try:
        export_split_image(image_path=image_path, output_path_base=output_path_base)
    except FileNotFoundError as e:
        ## this is ok for an incomplete test file
        pass